# Question A2 (10 marks)

### In this question, we will determine the optimal batch size for mini-batch gradient descent. Find the optimal batch size for mini-batch gradient descent by training the neural network and evaluating the performances for different batch sizes. Note: Use 5-fold cross-validation on training partition to perform hyperparameter selection. You will have to reconsider the scaling of the dataset during the 5-fold cross validation.

* note: some cells are non-editable and cannot be filled, but leave them untouched. Fill up only cells which are provided.

#### Plot mean cross-validation accuracies on the final epoch for different batch sizes as a scatter plot. Limit search space to batch sizes {128, 256, 512, 1024}. Next, create a table of time taken to train the network on the last epoch against different batch sizes. Finally, select the optimal batch size and state a reason for your selection.

This might take a while to run, so plan your time carefully.

In [38]:
import tqdm
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from scipy.io import wavfile as wav

from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

from common_utils import set_seed

# setting seed
set_seed()

2. To reduce repeated code, place your

- network (MLP defined in QA1)
- torch datasets (CustomDataset defined in QA1)
- loss function (loss_fn defined in QA1)

in a separate file called **common_utils.py**

Import them into this file. You will not be repenalised for any error in QA1 here as the code in QA1 will not be remarked.

The following code cell will not be marked.

In [39]:
# YOUR CODE HERE
from common_utils import MLP, CustomDataset, loss_fn, split_dataset
# from common_utils import split_dataset, preprocess_dataset

# def preprocess(df):
#     # YOUR CODE HERE
    
#     X_train, y_train, X_test, y_test = split_dataset(df, 'filename', 0.30, 1)
#     X_train_scaled, X_test_scaled = preprocess_dataset(X_train, X_test)
    
#     return X_train_scaled, y_train, X_test_scaled, y_test

# import the x train and y train datasets
df = pd.read_csv('simplified.csv')
df['label'] = df['filename'].str.split('_').str[-2]

df['label'].value_counts()
#change to train test split?
X_train, y_train, X_test, y_test = split_dataset(df, 'filename', 0.30, 1)
print(X_train)
print(y_train)

# X_train_scaled, y_train, X_test_scaled, y_test = preprocess(df)
# X_train, y_train = X_train_scaled, y_train

            tempo  total_beats  average_beats  chroma_stft_mean  \
5358    95.703125         1874     187.400000          0.567137   
642    103.359375          477      79.500000          0.549953   
7565    78.302557          875     125.000000          0.646271   
9584   112.347147         3430     201.764706          0.599859   
9374   198.768029         6870     214.687500          0.724747   
...           ...          ...            ...               ...   
7813   151.999081         3349     176.263158          0.591543   
10955  107.666016         3107     194.187500          0.514742   
905    161.499023        16138     375.302326          0.492115   
5192    92.285156          247      61.750000          0.526634   
235     95.703125          602      86.000000          0.500863   

       chroma_stft_var  chroma_cq_mean  chroma_cq_var  chroma_cens_mean  \
5358          0.088985        0.515726       0.076869          0.262738   
642           0.088597        0.488051       

3. Define different folds for different batch sizes to get a dictionary of training and validation datasets. Preprocess your datasets accordingly.

In [40]:
def generate_cv_folds_for_batch_sizes(parameters, X_train, y_train):
    """
    returns:
    X_train_scaled_dict(dict) where X_train_scaled_dict[batch_size] is a list of the preprocessed training matrix for the different folds.
    X_val_scaled_dict(dict) where X_val_scaled_dict[batch_size] is a list of the processed validation matrix for the different folds.
    y_train_dict(dict) where y_train_dict[batch_size] is a list of labels for the different folds
    y_val_dict(dict) where y_val_dict[batch_size] is a list of labels for the different folds
    """
    # YOUR CODE HERE
#     x train scaled and y train goes in here
#     x train scaled dict is the list of of 4/5 matrices
#     x val scaled dict is the last 1/5 matrix for testing
#     y train dict is the list of 4/5 labels
#     y val dict is the last 1/5 labels for testing
    
#     It will not differ by batch size. X_train_scaled_dict[128] is a list of train dataset for the different folds, and you should have 5 elements in the list in total. It is the same as X_train_scaled_dict[256], etc
#     X_train_scaled_dict should look like {128:[list of 5 folds] 256:[list of 5 folds], 512: [list of 5 folds], 1024: [list of 5 folds]}
#     y_train_dict is a dictionary of 4x5 elements as well, each element is the matrix of labels to train towards
    
#     customdataset = xtrain, ytrain
    
#     cv = KFold(n_splits=5, shuffle=True, random_state=1)
#     for train_idx, test_idx in cv.split(X_train, y_train):
#         X_train_scaled_dict, y_train_dict  = X_train[train_idx], y_train[train_idx]
#         X_val_scaled_dict, y__val_dict = X_train[test_idx], y_train[test_idx]
    n_splits = 5 
    batch_sizes = parameters  # Default to batch size of 32 if not provided
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)
    
    X_train_scaled_dict = {batch_size: [] for batch_size in batch_sizes}
    X_val_scaled_dict = {batch_size: [] for batch_size in batch_sizes}
    y_train_dict = {batch_size: [] for batch_size in batch_sizes}
    y_val_dict = {batch_size: [] for batch_size in batch_sizes}
    
    for train_idx, val_idx in kf.split(X_train):
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        standard_scaler = preprocessing.StandardScaler()
        X_train_fold_scaled = standard_scaler.fit_transform(X_train_fold)
        X_val_fold_scaled = standard_scaler.fit_transform(X_val_fold)
        
        for batch_size in batch_sizes:
            X_train_scaled_dict[batch_size].append(X_train_fold_scaled)
            X_val_scaled_dict[batch_size].append(X_val_fold_scaled)
            y_train_dict[batch_size].append(y_train_fold)
            y_val_dict[batch_size].append(y_val_fold)
    
    return X_train_scaled_dict, X_val_scaled_dict, y_train_dict, y_val_dict

batch_sizes = [128,256,512,1024]
X_train_scaled_dict, X_val_scaled_dict, y_train_dict, y_val_dict = generate_cv_folds_for_batch_sizes(batch_sizes, X_train.to_numpy(), y_train)
print(len(X_train_scaled_dict))
print(len(X_train_scaled_dict[128]))
# print(X_val_scaled_dict)
# print(y_train_dict)
# print(y_val_dict)

4
5


4. Perform hyperparameter tuning for the different batch sizes with 5-fold cross validation.

In [ ]:

# YOUR CODE HERE

batch_sizes = []
cross_validation_accuracies, cross_validation_times = find_optimal_hyperparameter(X_train_scaled_dict, X_val_scaled_dict, y_train_dict, y_val_dict, batch_sizes, 'batch_size')


5. Plot scatterplot of mean cross validation accuracies for the different batch sizes.

In [ ]:
# YOUR CODE HERE

6. Create a table of time taken to train the network on the last epoch against different batch sizes. Select the optimal batch size and state a reason for your selection.

In [ ]:
df = pd.DataFrame({'Batch Size':
                   'Last Epoch Time':
                  })

df

In [ ]:
# YOUR CODE HERE
optimal_batch_size =
reason =